In [7]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [8]:
def extract_sample_number(file_name):
    match = re.search(r'sample #(\d+)', file_name)
    if match:
        return int(match.group(1))
    # Return a large number to push files without sample numbers to the end
    return float('inf')

def data_SDR(path_folder):
    # List all files in the folder
    file_list = os.listdir(path_folder)

    # Sort files by sample number
    sorted_file_list = sorted(file_list, key=extract_sample_number)

    # Initialize data matrix based on the sorted file list
    exploration_path = os.path.join(path_folder, sorted_file_list[0])
    data_exploration = np.load(exploration_path)
    
    # Check if the data is complex
    is_complex = np.iscomplexobj(data_exploration)
    
    # Initialize the data matrix with the correct dtype
    data_matrix = np.zeros((len(sorted_file_list), data_exploration.shape[0]), dtype=complex if is_complex else float)

    # Load all files into data_matrix
    for pos, file_name in enumerate(sorted_file_list):
        full_path = os.path.join(path_folder, file_name)
        data = np.load(full_path)
        #print(f"Loading {file_name} (sample #{extract_sample_number(file_name)}), shape = {data.shape}")
        data_matrix[pos] = data

    print("Data matrix loading complete.")
    return data_matrix

In [9]:
path_file_folder = r'C:\Samples-Hack-RF'
data_matrix = data_SDR(path_file_folder)

r_signal = data_matrix[6]

fs = 20e6  # Frecuencia de muestreo (ajustar a la frecuencia de muestreo de tu SDR)
time_sample = 0.01  # Duración total en segundos
num_samples = int(fs * time_sample)
t = np.linspace(0, time_sample, num_samples)

Data matrix loading complete.


In [ ]:
f, Pxx = scipy.signal.welch(r_signal, fs=fs, nperseg=1024, scaling='density')
f, Pxx = scipy.signal.find_peaks(Pxx)